<a href="https://colab.research.google.com/github/LaoCabrales/reservas_hotel/blob/main/reserva_hotel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import datetime
import re
import time

# --- CONFIGURACIÓN SIMULADA ---
# Simulación de un calendario de reservas ocupadas.
# En un sistema real, esto se cargaría desde una base de datos (como Firestore o PostgreSQL).
RESERVAS_OCUPADAS = {
    'Cabaña 1': [
        (datetime.date(2025, 12, 20), datetime.date(2025, 12, 25)), # Ejemplo de reserva
    ],
    'Cabaña 2': [],
    'Camping': [
        (datetime.date(2025, 11, 1), datetime.date(2025, 11, 5)), # Ejemplo de reserva
    ]
}

EMAIL_HOTEL = "indika.hospedaje@gmail.com"

# --- DETALLES DE PAGO ELECTRÓNICO SIMULADO ---
DETALLES_PAGO = {
    '1': {
        'nombre': 'Bancolombia (Cuenta de Ahorros)',
        'info': 'Cuenta No. 123-456789-00 a nombre de INDIKA HOSPEDAJE'
    },
    '2': {
        'nombre': 'Nequi (Número de Celular)',
        'info': 'Número de celular: 300-1234567 (INDIKA HOSPEDAJE)'
    }
}

# --- FUNCIONES AUXILIARES ---

def validar_edad(edad):
    """Valida si el usuario es mayor de 18 años."""
    try:
        edad = int(edad)
        if edad < 18:
            return False, "Debe ser mayor de edad (18 años o más) para realizar una reserva."
        return True, edad
    except ValueError:
        return False, "La edad debe ser un número válido."

def validar_email(email):
    """Valida el formato básico de un correo electrónico."""
    # Expresión regular simple para validar un email
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if re.fullmatch(regex, email):
        return True, email
    return False, "Formato de correo electrónico no válido."

def validar_fecha(fecha_str):
    """Valida y convierte una cadena de texto a un objeto datetime.date."""
    try:
        # Se asume formato DD-MM-YYYY
        fecha = datetime.datetime.strptime(fecha_str, '%d-%m-%Y').date()
        if fecha < datetime.date.today():
            return False, "La fecha no puede ser en el pasado."
        return True, fecha
    except ValueError:
        return False, "Formato de fecha no válido. Use DD-MM-YYYY (Ej: 31-12-2025)."

def verificar_disponibilidad(tipo_reserva, fecha_inicio, fecha_fin):
    """
    Simula la verificación de disponibilidad para un rango de fechas.
    Retorna True si está disponible, False si está ocupado.
    """
    if fecha_inicio >= fecha_fin:
        print(" [!] La fecha de inicio debe ser anterior a la fecha de fin.")
        return False

    # Itera sobre las reservas ya existentes para el tipo de alojamiento
    for inicio_ocupado, fin_ocupado in RESERVAS_OCUPADAS.get(tipo_reserva, []):
        # Verifica si hay superposición entre el rango solicitado y los rangos ocupados
        # Superposición = (Inicio solicitado < Fin ocupado) AND (Fin solicitado > Inicio ocupado)
        if fecha_inicio < fin_ocupado and fecha_fin > inicio_ocupado:
            print(f" [!] Lo sentimos, la reserva se superpone con un período ya ocupado: {inicio_ocupado.strftime('%d-%m-%Y')} a {fin_ocupado.strftime('%d-%m-%Y')}.")
            return False

    return True

def enviar_email_simulado(destinatario, asunto, cuerpo):
    """Simula el envío de un correo electrónico imprimiendo el contenido."""
    print("\n" + "="*50)
    print(f"SIMULACIÓN DE ENVÍO DE CORREO A: {destinatario}")
    print(f"ASUNTO: {asunto}")
    print("="*50)
    print(cuerpo)
    print("="*50 + "\n")

def procesar_pago():
    """Simula el proceso de pago electrónico."""
    print("\n--- PROCESO DE PAGO ELECTRÓNICO ---")

    while True:
        print("Elija un método de pago:")
        for key, details in DETALLES_PAGO.items():
            print(f"   [{key}] {details['nombre']}")

        opcion_pago = input("   Ingrese el número de la opción de pago: ")

        if opcion_pago in DETALLES_PAGO:
            pago_elegido = DETALLES_PAGO[opcion_pago]
            print(f"\n[i] Ha seleccionado: {pago_elegido['nombre']}")
            print(f"    Detalle para realizar la transferencia: {pago_elegido['info']}")

            print("\n[!] En un sistema real, aquí se verificaría la transferencia.")
            confirmar_pago = input("¿Ha realizado la transferencia y desea confirmar el pago (s/n)? ").lower()

            if confirmar_pago == 's':
                print(" [√] Pago confirmado (simulado).")
                return pago_elegido['nombre']
            else:
                print(" [X] Pago no confirmado. Regresando a la reserva.")
                return None
        else:
            print(" [!] Opción de pago no válida.")


# --- FUNCIÓN PRINCIPAL DE RESERVA ---

def hacer_reserva():
    """Guía al usuario a través del proceso de reserva con validaciones."""

    print("--- SISTEMA DE RESERVAS INDIKA HOSPEDAJE ---")

    # 1. Recolección de Datos Personales
    nombre_completo = input("1. Nombre y Apellido: ")

    while True:
        edad_input = input("3. Edad: ")
        es_valida, resultado = validar_edad(edad_input)
        if es_valida:
            edad = resultado
            break
        print(f" [!] Error en Edad: {resultado}")

    identificacion = input("2. Número de Identificación: ")

    numero_celular = input("4. Número de Celular: ")

    while True:
        email_input = input("5. Correo Electrónico (Para resumen): ")
        es_valido, resultado = validar_email(email_input)
        if es_valido:
            email = resultado
            break
        print(f" [!] Error en Email: {resultado}")

    # 2. Selección de Reserva y Huéspedes
    tipos_validos = {
        '1': {'nombre': 'Cabaña 1', 'max_huespedes': 10},
        '2': {'nombre': 'Cabaña 2', 'max_huespedes': 6},
        '3': {'nombre': 'Camping', 'max_huespedes': 10},
    }

    while True:
        print("\n6. Tipo de Reserva:")
        print("   [1] Cabaña 1 (Max. 10 huéspedes)")
        print("   [2] Cabaña 2 (Max. 6 huéspedes)")
        print("   [3] Espacio para Camping (Max. 10 huéspedes)")
        opcion = input("   Ingrese el número de la opción: ")

        if opcion in tipos_validos:
            tipo_reserva = tipos_validos[opcion]['nombre']
            max_huespedes = tipos_validos[opcion]['max_huespedes']
            break
        print(" [!] Opción no válida. Por favor, ingrese 1, 2 o 3.")

    while True:
        try:
            huespedes = int(input(f"8. Número de Huéspedes (Mín. 1, Máx. {max_huespedes}): "))
            if 1 <= huespedes <= max_huespedes:
                break
            print(f" [!] El número de huéspedes debe estar entre 1 y {max_huespedes} para {tipo_reserva}.")
        except ValueError:
            print(" [!] Por favor, ingrese un número entero.")

    # 3. Selección y Validación de Fechas
    print("\n7. Días de Reserva (Formato DD-MM-YYYY):")
    while True:
        fecha_inicio_str = input("   Fecha de INICIO de la reserva (DD-MM-YYYY): ")
        es_valida_i, fecha_inicio = validar_fecha(fecha_inicio_str)
        if not es_valida_i:
            print(f" [!] Error en fecha de inicio: {fecha_inicio}")
            continue

        fecha_fin_str = input("   Fecha de FIN de la reserva (DD-MM-YYYY): ")
        es_valida_f, fecha_fin = validar_fecha(fecha_fin_str)
        if not es_valida_f:
            print(f" [!] Error en fecha de fin: {fecha_fin}")
            continue

        # Verificar disponibilidad simulada
        if verificar_disponibilidad(tipo_reserva, fecha_inicio, fecha_fin):
            print(" [√] ¡Fechas disponibles!")
            break

    # 4. Confirmación de la Reserva

    print("\n--- RESUMEN DE LA RESERVA ---")
    print(f" Cliente: {nombre_completo} (ID: {identificacion})")
    print(f" Edad: {edad} años")
    print(f" Contacto: {numero_celular} / {email}")
    print(f" Alojamiento: {tipo_reserva}")
    print(f" Huéspedes: {huespedes}")
    print(f" Período: {fecha_inicio.strftime('%d-%m-%Y')} hasta {fecha_fin.strftime('%d-%m-%Y')}")
    print("-" * 30)

    confirmar = input("¿Desea confirmar la reserva y proceder al pago? (si/no): ").lower()

    if confirmar == 'si':
        # --- PASO 5: PROCESAR PAGO ---
        metodo_pago = procesar_pago()

        if metodo_pago:
            # Si el pago es exitoso (simulado), se finaliza la reserva
            # 5. Procesar y Guardar Reserva (Simulado)
            RESERVAS_OCUPADAS[tipo_reserva].append((fecha_inicio, fecha_fin))

            # --- PREPARACIÓN DEL CONTENIDO DE EMAIL ---

            cuerpo_resumen = f"""
            Estimado/a {nombre_completo},

            ¡Su reserva ha sido confirmada exitosamente!

            A continuación, el detalle de su reserva:
            -------------------------------------------
            Tipo de Alojamiento: {tipo_reserva}
            Número de Huéspedes: {huespedes}
            Check-in: {fecha_inicio.strftime('%d-%m-%Y')}
            Check-out: {fecha_fin.strftime('%d-%m-%Y')}
            Método de Pago: {metodo_pago} (Pago confirmado)
            Datos de Contacto: {numero_celular} / {email}
            -------------------------------------------
            Agradecemos su preferencia. ¡Lo esperamos!
            """

            cuerpo_hotel = f"""
            NUEVA RESERVA REALIZADA

            Cliente: {nombre_completo} (ID: {identificacion})
            Email: {email}
            Celular: {numero_celular}
            Alojamiento: {tipo_reserva}
            Hués.: {huespedes}
            Período: {fecha_inicio.strftime('%d-%m-%Y')} a {fecha_fin.strftime('%d-%m-%Y')}
            Método de Pago: {metodo_pago}
            """

            # 6. Envío de Correos (Simulado)

            # Correo al usuario
            enviar_email_simulado(email, "Confirmación de Reserva y Pago - INDIKA HOSPEDAJE", cuerpo_resumen)
            # Correo al hotel
            enviar_email_simulado(EMAIL_HOTEL, f"NUEVA RESERVA (PAGADA) - {tipo_reserva} ({fecha_inicio.strftime('%d-%m-%Y')})", cuerpo_hotel)

            print(" [***] ¡Reserva y PAGO exitosos! Se ha enviado el resumen a su correo electrónico y al hotel.")
            print(f" [i] (La lista de reservas ocupadas ahora incluye: {RESERVAS_OCUPADAS[tipo_reserva]})")
        else:
            print("\n [X] Reserva NO finalizada. El pago no fue confirmado.")

    else:
        print(" [X] Reserva cancelada por el usuario.")

# Ejecución de la aplicación
if __name__ == "__main__":
    hacer_reserva()

    # Pequeña pausa para que se vea el resultado
    time.sleep(1)
    print("\nFin de la aplicación.")



--- SISTEMA DE RESERVAS INDIKA HOSPEDAJE ---
1. Nombre y Apellido: Laura Cabrales
3. Edad: 14
 [!] Error en Edad: Debe ser mayor de edad (18 años o más) para realizar una reserva.
3. Edad: 37
2. Número de Identificación: 621984654
4. Número de Celular: 54187635189
5. Correo Electrónico (Para resumen): laocabrales
 [!] Error en Email: Formato de correo electrónico no válido.
5. Correo Electrónico (Para resumen): laocabrales@gmail.com

6. Tipo de Reserva:
   [1] Cabaña 1 (Max. 10 huéspedes)
   [2] Cabaña 2 (Max. 6 huéspedes)
   [3] Espacio para Camping (Max. 10 huéspedes)
   Ingrese el número de la opción: 1
8. Número de Huéspedes (Mín. 1, Máx. 10): 12
 [!] El número de huéspedes debe estar entre 1 y 10 para Cabaña 1.
8. Número de Huéspedes (Mín. 1, Máx. 10): 6

7. Días de Reserva (Formato DD-MM-YYYY):
   Fecha de INICIO de la reserva (DD-MM-YYYY): 24-10-2025
   Fecha de FIN de la reserva (DD-MM-YYYY): 26-10-2025
 [√] ¡Fechas disponibles!

--- RESUMEN DE LA RESERVA ---
 Cliente: Laura Ca